In [2]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

In [3]:
import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text.lower(), flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []


In [4]:
wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article)
result = wiki.take(1)[0]

In [12]:
wiki.count()

4100

In [13]:
len(result)

10309

In [9]:
for word in result[:5]:
    print word

anarchism
anarchism
is
often
defined


In [11]:
sc.uiWebUrl

u'http://172.17.0.2:4040'

In [ ]:
# make func that finds target word and emits docs
# 